# Imports

In [1]:
%%javascript
var kernel = IPython.notebook.kernel;
var command = 
    "NOTEBOOK_NAME = " + 
    "'" + document.body.attributes['data-notebook-name'].value.slice(0, -6) + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
import os.path as op
import tempfile
import urllib.request

import numpy as np
import pandas as pd
import nglview

import kmbio

ModuleNotFoundError: No module named 'kmbio.PDB.Vector'

# PDB viewer

In [ ]:
initial_repr = [
    {"type": "cartoon", "params": {
        "sele": "protein", "color": "sstruc"
    }}
]

view = nglview.NGLWidget(background='white')

view.add_pdbid('4dkl')
view.background = 'black'
view.color_by('atomindex')
# view.parameters = {
#     # "percentages, "dist" is distance too camera in Angstrom
#     "clipNear": 0, "clipFar": 100, "clipDist": 10,
#     # percentages, start of fog and where on full effect
#     "fogNear": 0, "fogFar": 1000,
#     # background color
#     "backgroundColor": "white",
# }
# view.update_cartoon(opacity=0.4, component=0)
# view.camera = 'orthographic'
r = view.render_image()
print(r)

# Data

In [4]:
pdb_id = '2ou1'
cif_file = '/home/kimlab1/database_data/pdb/data/data/structures/divided/mmCIF/ou/2ou1.cif.gz' 
bioassembly_files = [
    '/home/kimlab1/database_data/pdb/data/data/structures/divided/mmCIF/ou/1ou0-1.cif',
]

In [5]:
import importlib
from kmtools import system_tools

In [6]:
with system_tools.decompress(cif_file) as input_file:
    sdict = kmbio.PDB.MMCIF2Dict(input_file)
    struct = kmbio.PDB.MMCIFParser(ignore_auth_id=True).get_structure(input_file)

Gunzipping file '/home/kimlab1/database_data/pdb/data/data/structures/divided/mmCIF/ou/2ou1.cif.gz'...
Gzipping the file back again...


NameError: name 'kmbio' is not defined

In [7]:
struct_0 = kmbio.PDB.MMCIFParser(ignore_auth_id=True).get_structure(bioassembly_files[0])

NameError: name 'kmbio' is not defined

In [19]:
list(struct[0])

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X']

In [24]:
list(struct_0[0])

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [123]:
gen = struct.get_chains().send(None)
gen

<Chain id=A>

In [124]:
_pdbx_struct_assembly_df = mmcif_key_to_dataframe(sdict, '_pdbx_struct_assembly')
_pdbx_struct_assembly_df.set_index('_pdbx_struct_assembly.id', inplace=True)
_pdbx_struct_assembly_df.head()

,_pdbx_struct_assembly.details,_pdbx_struct_assembly.method_details,_pdbx_struct_assembly.oligomeric_count,_pdbx_struct_assembly.oligomeric_details
_pdbx_struct_assembly.id,,,,
1,author_and_software_defined_assembly,PISA,2,dimeric
2,author_and_software_defined_assembly,PISA,2,dimeric
3,author_and_software_defined_assembly,PISA,2,dimeric
4,author_and_software_defined_assembly,PISA,2,dimeric
5,author_and_software_defined_assembly,PISA,2,dimeric


In [125]:
_pdbx_struct_assembly_gen_df = mmcif_key_to_dataframe(sdict, '_pdbx_struct_assembly_gen')
_pdbx_struct_assembly_gen_df.set_index('_pdbx_struct_assembly_gen.assembly_id', inplace=True)
_pdbx_struct_assembly_gen_df.head()

,_pdbx_struct_assembly_gen.asym_id_list,_pdbx_struct_assembly_gen.oper_expression
_pdbx_struct_assembly_gen.assembly_id,,
1,"A,B,M,N",1
2,"C,D,O,P",1
3,"E,F,Q,R",1
4,"G,H,S,T",1
5,"I,J,U,V",1


In [129]:
_pdbx_struct_assembly_gen_df.loc[['1', '2']]

,_pdbx_struct_assembly_gen.asym_id_list,_pdbx_struct_assembly_gen.oper_expression
_pdbx_struct_assembly_gen.assembly_id,,
1,"A,B,M,N",1
2,"C,D,O,P",1


In [127]:
# Additional data that is not needed now
# _pdbx_struct_assembly_prop_df = mmcif_key_to_dataframe(sdict, '_pdbx_struct_assembly_prop')
_pdbx_struct_oper_list_df = mmcif_key_to_dataframe(sdict, '_pdbx_struct_oper_list')
_pdbx_struct_oper_list_df['rotation'] = _pdbx_struct_oper_list_df.apply(get_rotation, axis=1)
_pdbx_struct_oper_list_df['translation'] = _pdbx_struct_oper_list_df.apply(get_translation, axis=1)
_pdbx_struct_oper_list_df.set_index('_pdbx_struct_oper_list.id', inplace=True)
_pdbx_struct_oper_list_df.head()

,_pdbx_struct_oper_list.matrix[1][1],_pdbx_struct_oper_list.matrix[1][2],_pdbx_struct_oper_list.matrix[1][3],_pdbx_struct_oper_list.matrix[2][1],_pdbx_struct_oper_list.matrix[2][2],_pdbx_struct_oper_list.matrix[2][3],_pdbx_struct_oper_list.matrix[3][1],_pdbx_struct_oper_list.matrix[3][2],_pdbx_struct_oper_list.matrix[3][3],_pdbx_struct_oper_list.name,_pdbx_struct_oper_list.symmetry_operation,_pdbx_struct_oper_list.type,_pdbx_struct_oper_list.vector[1],_pdbx_struct_oper_list.vector[2],_pdbx_struct_oper_list.vector[3],rotation,translation
_pdbx_struct_oper_list.id,,,,,,,,,,,,,,,,,
1,1.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"


In [26]:
# Additional data that is not needed now
# _pdbx_struct_assembly_prop_df = mmcif_key_to_dataframe(sdict, '_pdbx_struct_assembly_prop')
_pdbx_struct_assembly_gen_depositor_info = kmbio.PDB.MMCIFParser.mmcif_key_to_dataframe(sdict, '_pdbx_struct_assembly_gen_depositor_info')
_pdbx_struct_assembly_gen_depositor_info['rotation'] = _pdbx_struct_oper_list_df.apply(get_rotation, axis=1)
_pdbx_struct_oper_list_df['translation'] = _pdbx_struct_oper_list_df.apply(get_translation, axis=1)
_pdbx_struct_assembly_gen_depositor_info.set_index('_pdbx_struct_oper_list.id', inplace=True)
_pdbx_struct_assembly_gen_depositor_info.head()

AttributeError: type object 'MMCIFParser' has no attribute 'mmcif_key_to_dataframe'

In [63]:
for oper_list_id, (rotation, translation) in (
        _pdbx_struct_oper_list_df
        .set_index('_pdbx_struct_oper_list.id')
        [['rotation', 'translation']]
        .iterrows()):
    print(oper_list_id, rotation, translation)

1 [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]] [0.0, 0.0, 0.0]


In [142]:
assembly_id = '1'

asym_id_list = _pdbx_struct_assembly_gen_df.at[assembly_id, '_pdbx_struct_assembly_gen.asym_id_list']
oper_expression = _pdbx_struct_assembly_gen_df.at[assembly_id, '_pdbx_struct_assembly_gen.oper_expression']

structure = kmbio.PDB.Structure.Structure(pdb_id)

for i, oper in enumerate(oper_expression.split(',')):
    model = kmbio.PDB.Model.Model(i)
    structure.add(model)
    for asym_id in asym_id_list.split(','):
        chain = struct[0][asym_id].copy()
        model.add(chain)
    rotation = _pdbx_struct_oper_list_df.at[oper, 'rotation']
    translation = _pdbx_struct_oper_list_df.at[oper, 'translation']
    model.transform(rotation, translation)
    
io = kmbio.PDB.PDBIO()
io.set_structure(structure)
io.save('{}-{}.cif'.format(pdb_id, assembly_id), select=kmbio.PDB.Select())

In [143]:
!ls

2ou1-1.cif  explore_mmcif_bioassembly.html
2ou1.cif    explore_mmcif_bioassembly.ipynb


In [ ]:
new_str

In [114]:
        
for oper_list_id, (rotation, translation) in (
        _pdbx_struct_oper_list_df
        .set_index('_pdbx_struct_oper_list.id')
        [['rotation', 'translation']]
        .iterrows()):
    print(oper_list_id, rotation, translation)
    
    
    hetatm_chain = 

    
    df

TypeError: __call__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
assert len(_pdbx_struct_assembly_df) == len(_pdbx_struct_assembly_gen_df)


In [50]:
assert len(_pdbx_struct_assembly_df) == len(_pdbx_struct_assembly_df)
assert len(_pdbx_struct_assembly_gen_df) == len(_pdbx_struct_assembly_gen_df)

bioassembly_df = (
    _pdbx_struct_assembly_df
    .merge(
        _pdbx_struct_assembly_gen_df, 
        left_on='_pdbx_struct_assembly.id',
        right_on='_pdbx_struct_assembly_gen.assembly_id',
        how='inner')
)
bioassembly_df.head()

,_pdbx_struct_assembly.details,_pdbx_struct_assembly.id,_pdbx_struct_assembly.method_details,_pdbx_struct_assembly.oligomeric_count,_pdbx_struct_assembly.oligomeric_details,_pdbx_struct_assembly_gen.assembly_id,_pdbx_struct_assembly_gen.asym_id_list,_pdbx_struct_assembly_gen.oper_expression
0,author_and_software_defined_assembly,1,PISA,2,dimeric,1,"A,B,M,N",1
1,author_and_software_defined_assembly,2,PISA,2,dimeric,2,"C,D,O,P",1
2,author_and_software_defined_assembly,3,PISA,2,dimeric,3,"E,F,Q,R",1
3,author_and_software_defined_assembly,4,PISA,2,dimeric,4,"G,H,S,T",1
4,author_and_software_defined_assembly,5,PISA,2,dimeric,5,"I,J,U,V",1


'A,B,M,N'

In [ ]:
    bioassembly_df = (
        _pdbx_struct_assembly_df
        .merge(
            _pdbx_struct_assembly_gen_df, 
            left_on='_pdbx_struct_assembly.id',
            right_on='_pdbx_struct_assembly_gen.assembly_id',
            how='inner')
        .merge(
            _pdbx_struct_oper_list_df, 
            left_on='_pdbx_struct_assembly_gen.oper_expression',
            right_on='_pdbx_struct_oper_list.id',
            how='inner')
    )
    assert len(bioassembly_df) == len(_pdbx_struct_assembly_df)
    assert not (set(bioassembly_df['_pdbx_struct_assembly.id']) ^ 
                set(_pdbx_struct_assembly_df['_pdbx_struct_assembly.id']))
    return bioassembly_df

In [ ]:
for rotation, translation in 

In [43]:
get_bioassembly_data(sdict)

,_pdbx_struct_assembly.details,_pdbx_struct_assembly.id,_pdbx_struct_assembly.method_details,_pdbx_struct_assembly.oligomeric_count,_pdbx_struct_assembly.oligomeric_details,_pdbx_struct_assembly_gen.assembly_id,_pdbx_struct_assembly_gen.asym_id_list,_pdbx_struct_assembly_gen.oper_expression,_pdbx_struct_oper_list.id,_pdbx_struct_oper_list.matrix[1][1],...,_pdbx_struct_oper_list.matrix[3][2],_pdbx_struct_oper_list.matrix[3][3],_pdbx_struct_oper_list.name,_pdbx_struct_oper_list.symmetry_operation,_pdbx_struct_oper_list.type,_pdbx_struct_oper_list.vector[1],_pdbx_struct_oper_list.vector[2],_pdbx_struct_oper_list.vector[3],rotation,translation
0,author_and_software_defined_assembly,1,PISA,2,dimeric,1,"A,B,M,N",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
1,author_and_software_defined_assembly,2,PISA,2,dimeric,2,"C,D,O,P",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
2,author_and_software_defined_assembly,3,PISA,2,dimeric,3,"E,F,Q,R",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
3,author_and_software_defined_assembly,4,PISA,2,dimeric,4,"G,H,S,T",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
4,author_and_software_defined_assembly,5,PISA,2,dimeric,5,"I,J,U,V",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
5,author_and_software_defined_assembly,6,PISA,2,dimeric,6,"K,L,W,X",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
6,software_defined_assembly,7,PISA,4,tetrameric,7,"E,F,G,H,Q,R,S,T",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
7,software_defined_assembly,8,PISA,4,tetrameric,8,"A,B,C,D,M,N,O,P",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"
8,software_defined_assembly,9,PISA,4,tetrameric,9,"I,J,K,L,U,V,W,X",1,1,1.0000000000,...,0.0000000000,1.0000000000,1_555,"x,y,z",identity operation,0.0000000000,0.0000000000,0.0000000000,"[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[0.0, 0.0, 0.0]"


In [ ]:
[c for c in sdict if c.startswith('_struct_asym')]

In [ ]:
for key in relevant_keys:
    print(key, sdict.get(key, 'ERROR!!!'))

# Functions

In [ ]:
mmcif_field_to_dataframe(sdict, '_pdbx_struct_assembly')

In [ ]:
mmcif_field_to_dataframe(sdict, '_pdbx_struct_assembly_gen')

In [ ]:
mmcif_field_to_dataframe(sdict, '_pdbx_struct_assembly_prop')

In [ ]:
mmcif_field_to_dataframe(sdict, '_pdbx_struct_oper_list')

In [ ]:
bioassembly_columns = {
    '_pdbx_struct_assembly.id': 'id',
    '_pdbx_struct_assembly.details': 'details',
    '_pdbx_struct_assembly.method_details': 'method_details',
    '_pdbx_struct_assembly_gen.assembly_id': 'gen_assembly_id',
    '_pdbx_struct_assembly_gen.oper_expression': 'gen_oper_expression',
    '_pdbx_struct_assembly_gen.asym_id_list': 'gen_asym_id_list',
    '_pdbx_struct_assembly_prop.biol_id': 'prop_biol_id',
    '_pdbx_struct_assembly_prop.type': 'assembly_prop_type',
    '_pdbx_struct_assembly_prop.value': 'assembly_prop_value',
    '_pdbx_struct_assembly_prop.details': 'assembly_prop_details',
}


row = dict(
'gen': [],
'prop': []
)

def iter_bioassembly_data(sdict):
    for data in zip(*[sdict[c] for c in bioassembly_columns]):
        data_dict = dict(zip(bioassembly_columns.values(), data))
        assert data_dict['assembly_id'] == data_dict['assembly_gen_assembly_id']
        yield data_dict
        
        
list(iter_bioassembly_data(sdict))

In [ ]:
df = mmcif_field_to_dataframe(sdict, '_pdbx_struct_oper_list')
df

In [ ]:
df

In [ ]:
def parse_pdbx_struct_oper_list(sdict):
    df = mmcif_field_to_dataframe(sdict, '_pdbx_struct_oper_list')
    df['rotation'] = df.apply(get_rotation, axis=1)
    df['translation'] = df.apply(get_translation, axis=1)
    return df.select(lambda c: not c.endswith(']'), axis=1)

parse_pdbx_struct_oper_list(sdict)

In [ ]:
sdict_to_bioassembly(sdict)

In [ ]:
sdict_to_bioassembly(sdict)[1]['rotation']

In [ ]:
structure = kmbio.PDB.MMCIFParser().get_structure('4dkl', '4DKL.cif')

In [ ]:
structure.transform(
    sdict_to_bioassembly(sdict)[1]['rotation'],
    sdict_to_bioassembly(sdict)[1]['translation'],
)